In [ ]:
## Bring in the required libraries
library(tidyverse)
library(reshape2) 
library(cowplot) 

#  ***Question 1:***

Find the value of one year european call and put options on a stock
currently trading for \\$50 with a strike price of $50. 
With a risk free rate of 5% and the underlying stock volatility of 40% 


## Start by defining the terms

*s_0*  is the current stock price. 
*****************************************

*k* is the Current strike price. 
*****************************************
*r* is the Risk Free Rate. 
*****************************************
*sigma* is the volatility of underlying equity. 
*****************************************
*time* is the Time to maturity (in days). 

In [ ]:
D_1 <- function(s_0, k, r, sigma, time){
    
    ## Function to find d_1
    
    # Breaking the formula up into smaller parts 
    a <- log(s_0 / k) 
    b <- time * (r + ((sigma ^2 ) / 2))
    c <- sigma * sqrt(time) 
    
    #Put the parts of the equation together 
    d_1 <- (a + b) / (c) 
    return(d_1) 
}

d_1 <- D_1(s_0 = 50, k = 50, r = 0.05, sigma = 0.4, time = 1) 
paste0('The value of d_1 is: ',d_1) 

In [ ]:
D_2 <- function(s_0, k, r, sigma, time){
    
    ## Function to find d_2 
    
    # Breaking up the formula up into smaller parts 
    a <- log(s_0 / k) 
    b <- time * (r - ((sigma ^2 ) / 2)) 
    c <- sigma * sqrt(time) 
    
    # Put the parts of the equation together 
    d_2 <- (a + b) / (c) 
    return(d_2) 
}

d_2 <- D_2(s_0 = 50, k = 50, r = 0.05, sigma = 0.4, time = 1)
paste0('The value of d_2 is: ',d_2) 

In [ ]:
Call_Option <- function(k, s_0, d_1, r, time, d_2){
    
    ## Calculate the value of the call option 
    
    ## Breaking up the formula into smaller parts 
    a <- s_0 * pnorm(d_1)
    b <- k * exp(-r * time) * pnorm(d_2)
    c <- a - b
    return(c)     
}

call_option <- Call_Option(k = 50, s_0 = 50, d_1 = d_1, r = 0.05, time = 1, d_2 = d_2)
paste0('The value of the call option is: ', call_option) 

In [ ]:
Put_Option <- function(k, s_0, d_1, r, time, d_2){
    
    ## Calculate the value of the put option 
    
    ## Breaking up the formula into smaller parts 
    a <- s_0 * pnorm(-d_1) 
    b <- k * exp(-r * time) * pnorm(-d_2) 
    return( -a + b) 
}

put_option <- Put_Option(k = 50, s_0 = 50, d_1 = d_1, r = 0.05, time = 1, d_2 = d_2) 
paste0('The value of the put option is: ', put_option)  

# ***Question 2:*** 

Create a single function that can calculate the value of the call and put options. 
Output in a single variable

In [ ]:
Black_Scholes_Merton <- function(k, s_0, r, time, sigma){
    ### Use the previously created functions to create the Black Scholes Model
    
    ## Find the values for d_1 and d_2 
    d_1 <- D_1(s_0, k, r, sigma, time)
    d_2 <- D_2(s_0, k, r, sigma, time)
    
    ## Find the values for the call and put options 
    c <- Call_Option(k, s_0, d_1, r, time, d_2) 
    p <- Put_Option(k, s_0, d_1, r, time, d_2) 
    
    ## Return the call and put option values as a list 
    return(list('call' = c, 
               'put' = p))
}

## Use the Black Scholes Merton formula 
q_2 <- Black_Scholes_Merton(k = 50, s_0 = 50, r = 0.05, time = 1, sigma = 0.4)

paste0('The price of the call option is: ', q_2$call) 
paste0('The price of the put option is: ', q_2$put) 

# ***Question 3:***

Compute the value of the options for various strike prices ranging from 
40 to 60 in \\$0.10 increments

In [ ]:
### Start by defining the terms
s_0 <- 50 ## current stock price 
r <- 0.05 ## Risk Free Rate 
sigma <- 0.4 ## volatility of underlying equity 
time <- 1 ## Time to maturity (in days) 

In [ ]:
## Create a dataframe with K = the sequence of the strike price 
q_3 <- tibble('K' = seq(40, 60, 0.1))

## Add the call and put options to the dataframe 
q_3$Call <- Black_Scholes_Merton(q_3$K, 
                           s_0, r, time, sigma)$call
q_3$Put <- Black_Scholes_Merton(q_3$K, 
                           s_0, r, time, sigma)$put

head(q_3, 5)  

# ***Question 4:***
Call the function recursively in a for loop and save in a dataframe 
for the same range for the strike price 


In [ ]:


For_Loop_Black_Scholes <- function(s_0, r, time, sigma){
    ## Create a dataframe that has the K value, the Call option value, 
    ## and the Put option value
    
    df <- tibble(K = numeric(), 
              Call = numeric(), 
              Put = numeric())
    
    for (i in seq(40, 60, 0.1)){
        ## Recursively go through the strike price sequence 
        ## and find the call and put option values 
    
        ## Add row using tidyverse for both the call and the put options. 
        df <- df %>% add_row(K = i, 
                              Call = Black_Scholes_Merton(i, s_0, 
                                                         r, time, sigma)$call, 
                               Put = Black_Scholes_Merton(i, s_0, 
                                                         r, time, sigma)$put)
        
    }
     return(df)

}

q_4 <- For_Loop_Black_Scholes(s_0 = 50, 
                             r = 0.05, time = 1, sigma = 0.4) 
head(q_4) 

# ***Question 5:*** 

Repeat the function from question 4 but use the apply function

In [ ]:
q_5 <- tibble(K = seq(40, 60, 0.1) ) 

Black_Scholes_Merton_Call <- function(x){
    ## Find the call option value using the apply function
    return(Black_Scholes_Merton(k = x, s_0, r, time, sigma)$call)
}

Black_Scholes_Merton_Put <- function(x){
    ## Find the put option value using the apply function
    return(Black_Scholes_Merton(k = x, s_0, r, time, sigma)$put) 
}
q_5$Call <- sapply(q_5$K,Black_Scholes_Merton_Call)
q_5$Put <- sapply(q_5$K, Black_Scholes_Merton_Put)

head(q_5)

# ***Question 6:*** 
Check to ensure the following relation between the put 
and the call options holds: (The put-call parity) 


In [ ]:
Relation <- function(df, r, time, s_0) {
    ## Function to find put and call option parity
    
    df$Left_Relation <- df$Call + df$K * exp(-r * T) 
    df$Right_Relation <- df$Put + s_0 
    return(df) 
}

## Use the relation formula to find the values for the left and the right side of the 
## put-call parity formula
q_3_relation <- Relation(q_3, r = 0.05, time = 1, s_0 = 50)  
q_4_relation <- Relation(q_4, r = 0.05, time = 1, s_0 = 50) 
q_5_relation <- Relation(q_5, r = 0.05, time = 1, s_0 = 50) 

In [ ]:
Plot_Put_Call_Options <- function(df){
    ## Create a plot for the put and call Option values vs Strike Price
    
    ggplot(data = melt(df, id.var = 'K'), 
          aes(x = K, y = value, col = variable)) + 
        geom_line() + 
        labs(x = 'Strike Price K ($)', y = 'Option Value ($)', 
            colour = 'Variables', title = 'Strike Prices vs Option Prices') 
}

Plot_Put_Call_Parity <- function(df){
    ## Create plots for the put and call parity
    ## To see that the relation holds 
    
    ggplot(data = df, aes(x = Left_Relation, y = Right_Relation)) +
        geom_line() + 
        labs(x = 'Put Call Left Side', y = 'Put Call Right Side', 
             title = 'Put Call Parity') 
}

Once we have the values for the option prices (put and call) as well as the values for the right and the left hand sides of the put-call parity equation, we can then plot these. 

We are looking at the put-call parity relation to see whether or not the relation is held

In [ ]:
overall_title <- ggdraw() + 
    draw_label('Put Call Graphs- Option Value on Left Side, Parity Relation on Right')


q_3_plot <- plot_grid(Plot_Put_Call_Options(q_3), 
         Plot_Put_Call_Parity(q_3_relation))

q_4_plot <- plot_grid(Plot_Put_Call_Options(q_4), 
          Plot_Put_Call_Parity(q_4_relation))

q_5_plot <- plot_grid(Plot_Put_Call_Options(q_5), 
          Plot_Put_Call_Parity(q_5_relation))

plot_grid(overall_title, q_3_plot, q_4_plot, q_5_plot, ncol = 1, 
         labels = c('', 'Q_3', 'Q_4', 'Q_5'), label_size = 10)

As we can see in the graphs above, the relationship for the put-call parity holds
(comparing the left hand side of the equation with the right hand side of the equation).